In [1]:
from util import *
import torch
device = 'cpu'

In [2]:
def load_train_data():
    vocab = get_vocab()
    word2idx = {w:i for i,w in enumerate(vocab)}
    idx2word = {i:w for i,w in enumerate(vocab)}
    labels = []
    with open("data/ag_news_data/train.csv", 'r') as file:
        lines = file.readlines()[1:]
        for line in lines:
            labels.append(int(line.split(",")[-2]))
        lines = [",".join(line.split(",")[:-2]) for line in lines]
    train_data = []
    for i in range(len(lines)):
        lines[i] = extract(lines[i])[:SEQ_LENGTH]
        if len(lines[i]) < SEQ_LENGTH:
            lines[i].extend(['<pad>' for _ in range(SEQ_LENGTH - len(lines[i]))])
        label = labels[i]
        inputs = [word2idx[w] for w in lines[i]]
        targets = [1 if i == label else 0 for i in range(OUTPUT_SIZE)]
        train_data.append([inputs, label])
    return train_data, vocab
train_data, vocab = load_train_data()
    
    

In [3]:
class TextClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embedd = torch.nn.Embedding(
            num_embeddings=len(vocab),
            embedding_dim=EMBED_DIM
        )
        self.fc1 = torch.nn.Linear(EMBED_DIM*SEQ_LENGTH, HIDDEN_SIZE)
        self.fc2 = torch.nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
        self.acf1 = torch.nn.Sigmoid()
        self.acf2 = torch.nn.Softmax()
    def forward(self, x: torch.Tensor):
        x = self.embedd(x)
        x = x.flatten(1)
        x = self.acf1(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
model = TextClassifier()
train_data = train_data[:len(train_data)//24]
batch_size = len(train_data) 
n_batch = len(train_data)//batch_size
lossf = torch.nn.CrossEntropyLoss()
model.to(device)
n_epochs = 10
optimizer = torch.optim.Adam(model.parameters(),lr=1e-2)
for epoch in range(n_epochs):
    total_loss = 0
    for i in range(n_batch):
        optimizer.zero_grad()
        batch_data = train_data[i*batch_size:(i+1)*batch_size]
        inputs = torch.tensor([batch_data[i][0] for i in range(batch_size)]).to(device)
        targets = torch.tensor([batch_data[i][1] for i in range(batch_size)]).long().to(device)
        outputs = model(inputs)
        loss = lossf(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(total_loss)
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(n_batch):
            optimizer.zero_grad()
            batch_data = train_data[i*batch_size:(i+1)*batch_size]
            inputs = torch.tensor([batch_data[i][0] for i in range(batch_size)]).to(device)
            targets = torch.tensor([batch_data[i][1] for i in range(batch_size)]).long().to(device)
            outputs = model(inputs)
            loss = lossf(outputs, targets)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
        print(f"Accuracy : {correct/total}")


1.4263815879821777
Accuracy : 0.39
3.374135971069336
Accuracy : 0.4254
1.7485531568527222
Accuracy : 0.4652
1.3207236528396606
Accuracy : 0.4362
1.55916428565979
Accuracy : 0.4508
1.4312894344329834
Accuracy : 0.5312
1.135550856590271
Accuracy : 0.623
0.9274903535842896
Accuracy : 0.6836
0.8350181579589844
Accuracy : 0.6916
0.807162880897522
Accuracy : 0.6718
